Imports

In [6]:
from bs4 import BeautifulSoup
from random import randint
import requests
import urllib.request
import time
import pandas as pd
import os
import splitfolders

In [2]:
brands = ['chevrolet', 'renault', 'toyota', 'mazda', 'ford', 'kia', 'nissan', 'volkswagen', 'mercedes-benz', 'bmw']

Scrap the links of the images and store them in a `.csv` for each brand

In [3]:
class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0"


def get_attr(li, attr):
    list_attr = [ele[attr] for ele in li]
    return list_attr


def get_data(url):
    opener = AppURLopener()
    try:
        html = opener.open(url)
        soup = BeautifulSoup(html, 'html.parser')
        return soup, True
    except:
        randSleep = randint(60, 120)
        print(f'Access to the site denied, waiting {randSleep} seconds.')
        time.sleep(randSleep)
        return None, False


def image_link_scraping(brands):
    for brand in brands:
        list_ads = []
        success = False
        for idx in range(1, 500, 48): # Each page has 48 ads of cars
            while not(success):
                soup_page, success = get_data(f'https://carros.tucarro.com.co/{brand}/_Desde_{idx}_NoIndex_True')
            link_ads = soup_page.find_all('a', class_='ui-search-result__content ui-search-link')
            if link_ads != []:
                list_ads += get_attr(link_ads, 'href')
            success = False
        print(f'list of ads for {brand} created.')
        n = len(list_ads)
        data_vehic = []
        success = False
        i = 0
        c = 0
        while i < n:
            # Wait until we have access to the site
            while not(success):
                soup_annonce, success = get_data(list_ads[i]) 
            try:
                # We store 2 images of the same car
                img_1 = soup_annonce.find_all('figure', class_='ui-pdp-gallery__figure')[0].img['data-zoom']
                img_2 = soup_annonce.find_all('figure', class_='ui-pdp-gallery__figure')[randint(1,2)].img['data-zoom']
                data_vehic.append(img_1)
                data_vehic.append(img_2)
                # caract = soup_annonce.find_all('span', class_='andes-table__column--value')
                # We store the brand of the car
                # marque = caract[0].text
                # We store the type of the car
                # type = caract[-2].text
                #data_vehic.append((marque, type, img_1))
                #data_vehic.append((marque, type, img_2))
                if (c  % 200 == 0):
                    print(f'{c} links of ads have been scraped of {brand}.')
                c += 2
                i += 1
            except:
                i += 1
            success = False
        df = pd.DataFrame(data_vehic, columns=['img_url'])
        df.to_csv(f'./links-brand/{brand}.csv', sep=';', index=False)
        print(f'csv for {brand} has been created.')


In [3]:
image_link_scraping(brands)

/home/carlos/anaconda3/envs/data_science/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  del sys.path[0]


list of ads for chevrolet created.
0 links of ads have been scraped of chevrolet.
200 links of ads have been scraped of chevrolet.
Access to the site denied, waiting 63 seconds.
Access to the site denied, waiting 64 seconds.
Access to the site denied, waiting 105 seconds.
Access to the site denied, waiting 61 seconds.
400 links of ads have been scraped of chevrolet.
Access to the site denied, waiting 108 seconds.
Access to the site denied, waiting 62 seconds.
Access to the site denied, waiting 108 seconds.
600 links of ads have been scraped of chevrolet.
800 links of ads have been scraped of chevrolet.
Access to the site denied, waiting 104 seconds.
Access to the site denied, waiting 99 seconds.
Access to the site denied, waiting 85 seconds.
Access to the site denied, waiting 67 seconds.
csv for chevrolet has been created.
list of ads for renault created.
0 links of ads have been scraped of renault.
200 links of ads have been scraped of renault.
Access to the site denied, waiting 111 s

Download the images from the links in each `.csv` and store the images in a directory with the brand name

In [4]:
def download_imgs(brands):
    try:
        os.mkdir('./imgs-brand')
    except:
        pass
    for brand in brands:
        df = pd.read_csv(f'./links-brand/{brand}.csv', sep=';')
        urls = df['img_url']
        try:
            os.mkdir(f'./imgs-brand/{brand}')
        except:
            pass
        counter = 0
        for url in urls:
            try:
                file_name = f'./imgs-brand/{brand}/{brand}' + str(counter) + '.jpg'
                response = requests.get(url)
                file = open(file_name, 'wb')
                file.write(response.content)
                file.close()
                counter += 1
            except:
                pass
        print(f'{counter} images have successfully been downloaded for {brand}.')


In [5]:
download_imgs(brands)

962 images have successfully been downloaded for chevrolet.
1004 images have successfully been downloaded for renault.
956 images have successfully been downloaded for toyota.
988 images have successfully been downloaded for mazda.
1000 images have successfully been downloaded for ford.
972 images have successfully been downloaded for kia.
952 images have successfully been downloaded for nissan.
978 images have successfully been downloaded for volkswagen.
974 images have successfully been downloaded for mercedes-benz.
978 images have successfully been downloaded for bmw.


Create the test / train / val folders

In [10]:
def test_train_val():
    try:
        os.mkdir('./data')
    except:
        pass
    splitfolders.ratio(
        './imgs-brand/',
        output='./data',
        ratio=(.8, .1, .1))

In [11]:
test_train_val()

Copying files: 9764 files [25:06,  6.48 files/s]
